In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import r2_score, mean_squared_error

In [4]:
X_train=pd.read_csv('Splitted-Data/X_train_data.csv')
X_test=pd.read_csv('Splitted-Data/X_test.csv')
Y_train=pd.read_csv('Splitted-Data/Y_train.csv')
Y_test=pd.read_csv('Splitted-Data/Y_test.csv')

In [5]:
model=keras.Sequential([
    layers.Dense(128,activation='relu',input_shape=[X_train.shape[1]]),
    layers.BatchNormalization(),
    layers.Dropout(.2),
    layers.Dense(64,activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(32,activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(16,activation='relu'),
    layers.Dense(1,activation='linear')
])

/opt/conda/envs/sagemaker-distribution/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    optimizer='adam',
    loss='huber',
    metrics=['mae','mse']
)

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,617 (61.00 KB)

 Trainable params: 15,169 (59.25 KB)

 Non-trainable params: 448 (1.75 KB)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [12]:
early_stopping=EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    min_delta=0.001
)

In [13]:
redused_lr=ReduceLROnPlateau(
    monitor='val_loss',
    factor=.5,
    patience=10,
    verbose=1,
    min_lr=.0001
)

In [14]:
checkpoints=keras.callbacks.ModelCheckpoint(
    'Best_ANN_Model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [15]:
import tensorflow as tf
import datetime

In [16]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [17]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1  # Frequency (in epochs) to compute histograms
)

In [18]:
history=model.fit(
    X_train,Y_train,
    validation_data=(X_test,Y_test),
    batch_size=32,callbacks=[early_stopping,tensorboard_callback,checkpoints,redused_lr],epochs=100,verbose=1
)

Epoch 1/100
743/750 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.7519 - mae: 8.2123 - mse: 399.9164
Epoch 1: val_loss improved from inf to 3.33685, saving model to Best_ANN_Model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - loss: 7.7226 - mae: 8.1827 - mse: 399.4250 - val_loss: 3.3368 - val_mae: 3.6952 - val_mse: 2125.0667 - learning_rate: 0.0010
Epoch 2/100
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.0216 - mae: 3.4345 - mse: 258.3434
Epoch 2: val_loss improved from 3.33685 to 2.41953, saving model to Best_ANN_Model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 3.0215 - mae: 3.4343 - mse: 258.3401 - val_loss: 2.4195 - val_mae: 2.7265 - val_mse: 2043.1454 - learning_rate: 0.0010
Epoch 3/100
745/750 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.7146 - mae: 3.1187 - mse: 228.0242
Epoch 3: val_loss did not improve from 2.41953
750/750 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2.7138 - mae: 3.1179 - mse: 228.1076 - val_loss: 2.7316 - val_mae: 3.0264 - val_mse: 2031.0519 - 

In [19]:
y_pred_ann = model.predict(X_test)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [20]:
r2 = r2_score(Y_test, y_pred_ann)
rmse = np.sqrt(mean_squared_error(Y_test, y_pred_ann))

print(f"R2 Score: {r2:.4f}")
print(f"RMSE:     {rmse:.4f}")

R2 Score: 0.3374
RMSE:     40.4632


In [21]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir logs/fit

In [56]:
model.save('ANN_Model.keras')